In [ ]:
import pandas as pd
import polars as pl
import requests

In [ ]:
def pull_query(params: list, year: int) -> pl.DataFrame:
    # prepare custom census query
    param = ",".join(params)
    base = "https://api.census.gov/data/"
    flow = "/acs/acs5/profile"
    url = f"{base}{year}{flow}?get={param}&for=zip%20code%20tabulation%20area:*"
    df = pl.DataFrame(requests.get(url).json())

    # get names from DataFrame
    names = df.select(pl.col("column_0")).transpose()
    names = names.to_dicts().pop()
    names = dict((k, v.lower()) for k, v in names.items())

    # Pivot table
    df = df.drop("column_0").transpose()
    return df.rename(names).with_columns(year=pl.lit(year))

def pull_dp04() -> pl.DataFrame:
    empty_df = [
        pl.Series("total_units",[],dtype=pl.String),
        pl.Series("vacancy_rate",[],dtype=pl.String),
        pl.Series("median_owner_occupied_value",[],dtype=pl.String),
        pl.Series("median_rent_gross",[],dtype=pl.String),
        pl.Series("median_mortgage",[],dtype=pl.String),
        pl.Series("zipcode",[],dtype=pl.String),
        pl.Series("year",[],dtype=pl.Int32),
    ]
    df = pl.DataFrame(empty_df)
    for _year in range(2011, 2024):
        print(_year)
        tmp = pull_query(
            params=[
                "DP04_0001E",
                "DP04_0004E",
                "DP04_0089E",
                "DP04_0134E",
                "DP04_0101E"
            ],
            year=_year,
        )
        tmp = tmp.rename(
            {
                "dp04_0001e": "total_units",
                "dp04_0004e": "vacancy_rate",
                "dp04_0089e": "median_owner_occupied_value",
                "dp04_0134e": "median_rent_gross",
                "dp04_0101e": "median_mortgage"
            }
        )
        if _year < 2020:
            tmp = tmp.rename({"zip code tabulation area": "zipcode"}).drop(["state"])
        else:
            tmp = tmp.rename({"zip code tabulation area": "zipcode"})
        df = pl.concat([df,tmp], how="vertical")
    return df

In [ ]:
# df = pull_query([
#                 "DP04_0001E",
#                 "DP04_0004E",
#                 "DP04_0089E",
#                 "DP04_0134E",
#                 "DP04_0101E"
#             ], 2024)
# df 

In [ ]:
# df.drop(["state"]).drop(["state"])

In [ ]:
df = pull_dp04()
df

In [ ]:
df.write_csv("dp04.csv")